In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [14]:
training_data = pd.read_csv('train.csv')

In [15]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import spacy

In [16]:

punctuations = string.punctuation
import en_core_web_sm

nlp = en_core_web_sm.load()
stop_words = spacy.lang.en.stop_words.STOP_WORDS

parser = English()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations]
    mytokens = [ word for word in mytokens if word.isalnum()]
    # return preprocessed list of tokens
    return mytokens

In [17]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [18]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))


In [19]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)


In [20]:
from sklearn.model_selection import train_test_split

X = training_data['keyword'].fillna('') +' '+ training_data['location'].fillna('') +' '+\
    training_data['text'].fillna('')
ylabels = training_data.target # the labels, or answers, we want to test against

x_train, x_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [21]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier()

In [22]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x00000239065BF088>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x000002397F2B3828>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_

In [23]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(x_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.7854640980735552
Logistic Regression Precision: 0.7905405405405406
Logistic Regression Recall: 0.6978131212723658


In [24]:
pipe.fit(training_data.text,training_data.target)
predicted = pipe.predict(x_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9185639229422067
Logistic Regression Precision: 0.8957528957528957
Logistic Regression Recall: 0.9224652087475149


In [25]:
test_data = pd.read_csv('test.csv')
test_id = test_data.id
test_data['text'] = test_data['keyword'].fillna('') +' '+ test_data['location'].fillna('') +' '+\
                    test_data['text'].fillna('')

In [26]:
predicted = pipe.predict(test_data.text)

In [27]:
result = pd.DataFrame({'id':test_id,'target':predicted})
result.to_csv('submission.csv',index=False)

#### Accuracy :---
#####                - Local Accuracy : 79.11 (over split training data)
#####                - Online Accuracy : 79.44 (After fitting over split test data)
#####                - Online Accuracy : 76.99 (After fitting over all training data)